In [2]:
import torch
from torch.utils.data.dataset import Dataset
from PIL import Image
from os import path
from torchvision import transforms
import numpy as np

# First test implementation

## Costruction of dataset

- `all_labels.txt`: All image's labels of all entire dataset
- `classes.txt`: classes
- `test.txt`: test dataset
- `test.txt`: train dataset 

In [3]:
class TrashbeanDataset(Dataset):
    def __init__(self, base_path, txt_list, transform_dim=None):

        self.base_path = base_path
        self.images = np.loadtxt(txt_list, dtype=str, delimiter=',')

        # transforms.Resize the input image to the given size. If the image is torch Tensor, it is expected to             have […, H, W] shape, where … means an arbitrary number of leading dimensions
        self.transform = transforms.Compose([transforms.Resize(transform_dim), transforms.ToTensor()])

    def __getitem__(self, index):
        # recuperiamo il path dell'immagine di indice index e la relativa etichetta
        f,c = self.images[index]

        # carichiamo l'immagine utilizzando PIL
        im = Image.open(path.join(self.base_path, f))

        # se la trasformazione è definita, applichiamola all'immagine
        if self.transform is not None:
            im = self.transform(im)

        # converto l'etichetta in intero
        label = int(c)

        #restituisco un dizionario contenente immagine etichetta
        return {'image': im, 'label': label}

    # restituisce numero di campioni: la linghezza della lista images
    def __len__(self):
        return len(self.images)

In [4]:
dataset_train = TrashbeanDataset('static/datasets/img', 'static/datasets/train.txt', transform_dim=100) # can put (w,h) too

# dataset_test

sample = dataset_train[7]
# print(sample['image'].shape, sample['label'])
# print(dataset__.__len__())
# print(dataset__.__getitem__(0))

print(dataset_train[0]['image'].shape)


torch.Size([3, 177, 100])
